# Getting the Response From ESPN API and Converting it into a JSON file

In [1]:
# Import a library that allows to make HTTP request
import requests
import json
# Set the API endpoint
url = "https://hs-consumer-api.espncricinfo.com/v1/pages/matches/current?lang=en&latest=true"

# Use the library to perform an HTTP GET request to the URL
response = requests.get(url)
response_dict = json.loads(response.text)

# Function to clear Html tags


In [2]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

# Generating Summary of Ongoing Matches

In [3]:
summary_msg = []
summary_msg.append("---CURRENT GAMES IN LIST---")
for match_dict in response_dict['matches']:
    match_details = match_dict['slug'].upper()
    match_status = match_dict['statusText']
    summary_msg.append("---")
    summary_msg.append(match_details)
    summary_msg.append(match_status)
    try:
        teams={}
        toss ={}
        toss.update({1 : "Bat"})
        toss.update({2 : "Bowl"})
        for team in match_dict['teams']:
            team_name = team['team']['name']
            team_id = team['team']['id']
            teams.update({team_id : team_name})
        tossId = match_dict['tossWinnerTeamId']
        fieldId = match_dict['tossWinnerChoice']
        toss_status = "{} have won the toss and decided to {} first".format(teams[tossId],toss[fieldId])
        summary_msg.append(toss_status)
    except:
        pass
    summary_msg.append("---")


In [4]:
for msg in summary_msg:
    print(msg)

---CURRENT GAMES IN LIST---
---
GLAMORGAN-VS-LEICESTERSHIRE
Day 3 - Leics chose to field.
Leics have won the toss and decided to Bowl first
---
---
SUSSEX-VS-DERBYSHIRE
Day 3 - Sussex lead by 56 runs.
Derbyshire have won the toss and decided to Bowl first
---
---
WORCESTERSHIRE-VS-YORKSHIRE
Day 3 - WORCS trail by 203 runs.
WORCS have won the toss and decided to Bowl first
---
---
DURHAM-VS-GLOUCESTERSHIRE
Day 3 - Gloucs trail by 22 runs.
Gloucs have won the toss and decided to Bat first
---
---
SURREY-VS-NOTTINGHAMSHIRE
Day 3 - Surrey lead by 46 runs.
Notts have won the toss and decided to Bowl first
---
---
MIDDLESEX-VS-NORTHAMPTONSHIRE
Day 3 - Northants lead by 150 runs.
Northants have won the toss and decided to Bat first
---
---
KENT-VS-WARWICKSHIRE
Day 3 - Kent trail by 155 runs.
Kent have won the toss and decided to Bat first
---
---
SOMERSET-VS-HAMPSHIRE
Day 3 - Hampshire trail by 230 runs.
Somerset have won the toss and decided to Bat first
---
---
LANCASHIRE-VS-ESSEX
Day 3 - E

# Getting The IDs of Current Games

In [5]:
game_ids = []
for match_dict in response_dict['matches']:
    if match_dict['coverage']=="Y":
        series_id = match_dict['series']['objectId']
        match_id = match_dict['objectId']
        game_ids.append((series_id,match_id))

In [6]:
game_ids

[(1347098, 1347181),
 (1347098, 1347182),
 (1347098, 1347183),
 (1347098, 1347184),
 (1347099, 1347244),
 (1347099, 1347245),
 (1347099, 1347246),
 (1347099, 1347247),
 (1347099, 1347248),
 (1384571, 1384580),
 (1377001, 1377014),
 (1381943, 1382164),
 (1347396, 1347430),
 (1347396, 1347431),
 (1347396, 1347432),
 (1347396, 1347433),
 (1381384, 1381389),
 (1379754, 1379762),
 (1336063, 1336077),
 (1379595, 1379627),
 (1381201, 1381212),
 (1384571, 1384581),
 (1381943, 1382165),
 (1357742, 1381361)]

# Generating Messages for the top 3 games

In [7]:
game_msgs = []
game_msgs.append(["-----------------------------------------------------------------------------------------"])
game_msgs.append(["-------------------------------------NEW UPDATES ----------------------------------------"])
Batsmen = []
Bowlers = []
for game_id in game_ids[:3]:
    final_msgs=[]
    final_msgs.append("--------------------------------------------------")
    # Set the API endpoint
    url = "https://hs-consumer-api.espncricinfo.com/v1/pages/match/details?lang=en&seriesId={}&matchId={}&latest=true".format(game_id[0],game_id[1])

    # Use the library to perform an HTTP GET request to the URL
    response = requests.get(url)
    response_dict = json.loads(response.text)
    
    # Generating Initial Message
    match_details = response_dict['match']['slug'].upper()
    match_status = response_dict['match']['statusText']
    final_msgs.append(match_details)
    final_msgs.append(match_status)
    final_msgs.append("Status: " + response_dict['match']['status'])
    # Getting the current score
   
    curr_score = response_dict['match']['teams'][0]['score']
    country = response_dict['match']['teams'][0]['team']['name']
    country_2 = response_dict['match']['teams'][1]['team']['name']
    curr_score_2 = response_dict['match']['teams'][1]['score']
    msg_1 = "{}'s score is: {}".format(country, curr_score)
    msg_2 = "{}'s score is: {}".format(country_2, curr_score_2)
    msg_1 = msg_1.replace("&","and")
    msg_2 = msg_2.replace("&","and")
    final_msgs.append(msg_1)  
    final_msgs.append(msg_2)
    # Getting the recent commentaries
    i = 0
    try:
        for comments in response_dict['recentBallCommentary']['ballComments']:
            if comments['isFour'] or comments['isSix'] or comments['isWicket'] or i==0:
                overs = comments['oversActual']
                overnum = comments['overNumber']
                ballnum = comments['ballNumber']
                innings = comments['inningNumber']
                msg_2 = "Innings no:{} and {} overs have been bowled".format(innings, overs)
                final_msgs.append(comments['title'])
                if i == 0:
                    final_msgs.append(msg_2)
                if comments['isFour']:
                    final_msgs.append("Its a Boundary --4--")
                if comments['isSix']:
                    final_msgs.append("Its a maximum --6--")
                if comments['isWicket']:
                    final_msgs.append("Wicket down!")
                    final_msgs.append(comments['dismissalText']['commentary'])
                
                try:
                    final_msgs.append(comments['commentTextItems'][0]['html'])
                except:
                    pass
                if(i==0):
                    final_msgs.append("        RECENT HIGHLIGHTS         ")
                i+=1
        runs = ""
        for comments in response_dict['recentBallCommentary']['ballComments']:
            runs += str(comments['totalRuns']) + " "
            if comments['ballNumber'] == 6:
                runs += "|" + " "
        final_msgs.append(runs)
        
        for comments in response_dict['recentBallCommentary']['ballComments']:
            if comments['over'] != None:
                batsman_1 = comments['over']['overEndBatsmen'][0]['player']['longName']
                batsman_2 = comments['over']['overEndBatsmen'][1]['player']['longName']
                bowler_1 = comments['over']['overEndBowlers'][0]['player']['longName']
                bowler_2 = comments['over']['overEndBowlers'][1]['player']['longName']
                batsmen = "Batsmen in crease: {} and {}".format(batsman_1,batsman_2)
                bowlers = "Current Bowlers: {} and {}".format(bowler_1, bowler_2)
                final_msgs.append(batsmen)
                final_msgs.append(bowlers)
                Batsmen.append(country+ " vs " + country_2 +": " + batsmen)
                Bowlers.append(country+ " vs " + country_2 + ": " + bowlers)
                break
                
    except:
        pass
    # Clearing HTML
    for i in range(len(final_msgs)):
        final_msgs[i] = cleanhtml(final_msgs[i])
    final_msgs.append("-------------------------------------------------")
    
    game_msgs.append(final_msgs)

In [8]:
for game in game_msgs:
    for msg in game:
        print(msg)


-----------------------------------------------------------------------------------------
-------------------------------------NEW UPDATES ----------------------------------------
--------------------------------------------------
GLAMORGAN-VS-LEICESTERSHIRE
Day 3 - Leics chose to field.
Status: Live
Glamorgan's score is: 221/8
Leics's score is: None
Parkinson to Neser
Innings no:1 and 69.4 overs have been bowled
        RECENT HIGHLIGHTS         
Scriven to Harris
Wicket down!
James Harris lbw b Scriven 47 (112b 5x4 0x6 127m) SR: 41.96
0 0 0 0 0 | 0 0 0 0 0 0 | 1 2 0 0 0 0 | 1 0 1 
-------------------------------------------------
--------------------------------------------------
SUSSEX-VS-DERBYSHIRE
Day 3 - Sussex lead by 56 runs.
Status: Live
Sussex's score is: 402 and 61/3
Derbyshire's score is: 407
Scrimshaw to Carter
Innings no:3 and 17.2 overs have been bowled
        RECENT HIGHLIGHTS         
Scrimshaw to Carter
Its a Boundary --4--
1 0 0 | 0 1 0 1 0 4 | 1 0 0 0 0 1 | 0 0 3 1

In [9]:
import requests
import numpy as np
import pandas as pd
import html
import cv2
import urllib.parse
import time

# Bot Handling - Getting the Chat IDs from latest Requests

# Sending The Messages to Users using Telegram API

In [10]:
Bowlers

['Sussex vs Derbyshire: Current Bowlers: Sam Conners and George Scrimshaw',
 'Yorkshire vs WORCS: Current Bowlers: Mark Steketee and Dan Moriarty']

In [11]:

session = requests.Session()
session.headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.1.2222.33 Safari/537.36",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}

def telegram_bot_sendText(chat_id, text_message):
    bot_token = '6018192433:AAFPKYQBv6wCc-YT2Ce5OQ9tqbRqpNj0vus'
    responses = []
    bot_chatid = str(chat_id)
    if text_message == None:
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatid + \
                 '    &parse_mode=HTML&text=' + "none"
    else:
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatid + \
                 '    &parse_mode=HTML&text=' + text_message
    response = session.get(send_text, timeout= 10000, allow_redirects = False)
    responses.append(response.json())
    return responses

In [12]:
bot_url = "https://api.telegram.org/bot6018192433:AAFPKYQBv6wCc-YT2Ce5OQ9tqbRqpNj0vus/getUpdates"
response = session.get(bot_url)
response_dict = json.loads(response.text)
latest_id = pd.read_csv('lastId.csv').values[0][0]
print(latest_id)
print(response_dict)

for update_dict in response_dict['result']:
    if (update_dict['message']['text'] == "/summary" and update_dict['update_id'] > latest_id):
        chat_id = update_dict['message']['chat']['id']
        for msg in summary_msg:
            telegram_bot_sendText(chat_id, msg)
    elif (update_dict['message']['text'] == "/highlights" and update_dict['update_id'] > latest_id):
        chat_id = update_dict['message']['chat']['id']
        for game in game_msgs:
            for msg in game:
                telegram_bot_sendText(chat_id, msg)
    elif (update_dict['message']['text'] == "/score" and update_dict['update_id'] > latest_id):
        chat_id = update_dict['message']['chat']['id']
        for game in game_msgs:
            for msg in game:
                telegram_bot_sendText(chat_id, msg)
    elif (update_dict['message']['text'] == "/batsmen" and update_dict['update_id'] > latest_id):
        chat_id = update_dict['message']['chat']['id']
        for batsmen in Batsmen:
            telegram_bot_sendText(chat_id, batsmen)
    elif (update_dict['message']['text'] == '/bowlers' and update_dict['update_id'] > latest_id):
        chat_id = update_dict['message']['chat']['id']
        for bowler in Bowlers:
            telegram_bot_sendText(chat_id, bowler)   
    last_id = update_dict['update_id']
    
last_id = pd.DataFrame(np.array([last_id]))
print(last_id)
last_id.to_csv("lastId.csv", index=False)

733807506
{'ok': True, 'result': [{'update_id': 733807476, 'message': {'message_id': 2, 'from': {'id': 6318819299, 'is_bot': False, 'first_name': 'Jai', 'last_name': 'Ganesh', 'language_code': 'en'}, 'chat': {'id': 6318819299, 'first_name': 'Jai', 'last_name': 'Ganesh', 'type': 'private'}, 'date': 1689157014, 'text': 'hello'}}, {'update_id': 733807477, 'message': {'message_id': 3, 'from': {'id': 6318819299, 'is_bot': False, 'first_name': 'Jai', 'last_name': 'Ganesh', 'language_code': 'en'}, 'chat': {'id': 6318819299, 'first_name': 'Jai', 'last_name': 'Ganesh', 'type': 'private'}, 'date': 1689158483, 'text': 'summary'}}, {'update_id': 733807478, 'message': {'message_id': 4, 'from': {'id': 6318819299, 'is_bot': False, 'first_name': 'Jai', 'last_name': 'Ganesh', 'language_code': 'en'}, 'chat': {'id': 6318819299, 'first_name': 'Jai', 'last_name': 'Ganesh', 'type': 'private'}, 'date': 1689158706, 'text': 'summary'}}, {'update_id': 733807479, 'message': {'message_id': 5, 'from': {'id': 63188

           0
0  733807507


In [13]:
summary_msg

['---CURRENT GAMES IN LIST---',
 '---',
 'GLAMORGAN-VS-LEICESTERSHIRE',
 'Day 3 - Leics chose to field.',
 'Leics have won the toss and decided to Bowl first',
 '---',
 '---',
 'SUSSEX-VS-DERBYSHIRE',
 'Day 3 - Sussex lead by 56 runs.',
 'Derbyshire have won the toss and decided to Bowl first',
 '---',
 '---',
 'WORCESTERSHIRE-VS-YORKSHIRE',
 'Day 3 - WORCS trail by 203 runs.',
 'WORCS have won the toss and decided to Bowl first',
 '---',
 '---',
 'DURHAM-VS-GLOUCESTERSHIRE',
 'Day 3 - Gloucs trail by 22 runs.',
 'Gloucs have won the toss and decided to Bat first',
 '---',
 '---',
 'SURREY-VS-NOTTINGHAMSHIRE',
 'Day 3 - Surrey lead by 46 runs.',
 'Notts have won the toss and decided to Bowl first',
 '---',
 '---',
 'MIDDLESEX-VS-NORTHAMPTONSHIRE',
 'Day 3 - Northants lead by 150 runs.',
 'Northants have won the toss and decided to Bat first',
 '---',
 '---',
 'KENT-VS-WARWICKSHIRE',
 'Day 3 - Kent trail by 155 runs.',
 'Kent have won the toss and decided to Bat first',
 '---',
 '---',
